In [70]:
import pandas as pd
import numpy as np
import requests
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
#must pip install requests and selenium libraries in order for this to work

In [51]:
browser= webdriver.Chrome("chromedriver")
#download chromedriver.exe from https://chromedriver.storage.googleapis.com/index.html?path=74.0.3729.6/ and store in
#project folder

In [243]:
browser.get("https://open.spotify.com/playlist/0ScWlJ7pCqtG3DAUKo9JeO")
#put URL of spotify playlist here

In [244]:
html=browser.page_source
#get html source code for webpage

In [245]:
elem=html.split("tracklist-name")
#parse and filter source code for desired data such as title,artist,duration,year,etc.

In [246]:
import html
name=html.unescape(elem[1].split("dir=\"auto\">")[1].split("</div>")[0])
#html.unescape removes/fixes all html formatted text such as replacing '&amp;' with '&' etc.
name

'Building a Ladder'

In [247]:
artist=html.unescape(elem[1].split("tracklist-row__artist-name-link")[1].split("\">")[1].split("</a>")[0])
artist

'Hiatus Kaiyote'

In [248]:
album=html.unescape(elem[1].split("tracklist-row__album-name-link")[1].split("\">")[1].split("</a>")[0])
album

'Choose Your Weapon'

In [249]:
duration=elem[1].split("tracklist-duration tracklist-top-align\"><span>")[1].split("<")[0]
duration

'5:42'

In [250]:
albumurl="https://open.spotify.com"+elem[1].split("class=\"tracklist-row__album-name-link\" href=\"")[1].split("\">")[0]
albumurl

'https://open.spotify.com/album/3qzmmmRmVBiOuMvrerfW4z'

In [251]:
artisturl="https://open.spotify.com"+elem[1].split("href=\"")[1].split("\">")[0]+"/about"
artisturl

'https://open.spotify.com/artist/43JlwunhXm1oqdKyOa2Z9Y/about'

In [252]:
link=requests.get(albumurl)
source=link.text
#visit album webpage to get data regarding year of release

In [253]:
year=source.split("music:release_date\" content=\"")[1][0:4]
year

'2015'

In [254]:
link=requests.get(artisturl)
source=link.text
#visit artist webpage to get data regarding number of monthly listeners, location(city) of highest number of listeners

In [255]:
popularcity=source.split("<span class=\"horizontal-list__item__title\" dir=\"auto\">")[1].split("<")[0]
popularcity
#city where most listeners live

'London, GB'

In [256]:
cityfollowers=source.split("<span class=\"horizontal-list__item__subtitle\">")[1].split()[0].replace(",","")
cityfollowers
#number of listeners for this artist from this city

'19635'

In [257]:
monthlylisteners=source.split("Monthly Listeners: ")[1].split(",")[0].replace(",","")
monthlylisteners
#number of listeners for this artist per month

'586426'

In [258]:
import html
songs=[]
for x in elem[1:]:
    name=html.unescape(x.split("dir=\"auto\">")[1].split("</div>")[0])
    artist=html.unescape(x.split("tracklist-row__artist-name-link")[1].split("\">")[1].split("</a>")[0])
    album=html.unescape(x.split("tracklist-row__album-name-link")[1].split("\">")[1].split("</a>")[0])
    duration=x.split("tracklist-duration tracklist-top-align\"><span>")[1].split("<")[0]
    albumurl="https://open.spotify.com"+x.split("class=\"tracklist-row__album-name-link\" href=\"")[1].split("\">")[0]
    artisturl="https://open.spotify.com"+x.split("href=\"")[1].split("\">")[0]+"/about"
    playlist=html.unescape(elem[0].split("content=\"")[1].split(",")[0])
    playlistby=html.unescape(elem[0].split(", a playlist by ")[1].split(" on Spotify\">")[0])
    
    link=requests.get(albumurl)
    source=link.text
    
    year=source.split("music:release_date\" content=\"")[1][0:4].replace(",","")
    
    link=requests.get(artisturl)
    source=link.text
    
    if "Monthly Listeners:" in source:
        monthlylisteners=source.split("Monthly Listeners: ")[1].split(",")[0].replace(",","")
        popularcity=source.split("<span class=\"horizontal-list__item__title\" dir=\"auto\">")[1].split("<")[0]
        cityfollowers=source.split("<span class=\"horizontal-list__item__subtitle\">")[1].split()[0].replace(",","")
    
    
    
        songs.append([name,artist,album,duration,int(year),int(monthlylisteners),popularcity,int(cityfollowers),playlist,playlistby])

In [259]:
#convert songs list into dataframe
df= pd.DataFrame(songs,columns=["title","artist","album","duration","year","monthlylisteners","popularcity","cityfollowers","playlist","playlistby"])

In [260]:
df.head()

,title,artist,album,duration,year,monthlylisteners,popularcity,cityfollowers,playlist,playlistby
0,Building a Ladder,Hiatus Kaiyote,Choose Your Weapon,5:42,2015,586426,"London, GB",19635,Building a Ladder – Hiatus Kaiyote,ännas
1,You Keep Me Waiting,Snoh Aalegra,FEELS,4:27,2017,1224068,"Los Angeles, US",44864,Building a Ladder – Hiatus Kaiyote,ännas
2,Red Clay,Charlotte Dos Santos,Red Clay,2:20,2017,143505,"London, GB",7590,Building a Ladder – Hiatus Kaiyote,ännas
3,Dirty Hercules (feat Nai Palm),Ngaiire,Lamentations,4:00,2013,61972,"Melbourne, AU",3257,Building a Ladder – Hiatus Kaiyote,ännas
4,Molasses,Hiatus Kaiyote,Choose Your Weapon,4:49,2015,586426,"London, GB",19635,Building a Ladder – Hiatus Kaiyote,ännas


In [261]:
#check to see if datafile already exists
try:
    df2= pd.read_csv("spotify.csv")
except:
    print("No File")
    df2= pd.DataFrame()

In [262]:
#union between old data and new data
dftotal= pd.concat([df, df2])

In [263]:
#store combined data in csv file
dftotal.to_csv("spotify.csv",index=False)